<a href="https://colab.research.google.com/github/tousifo/ml_notebooks/blob/main/oaisis_3_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

# Load the two CSV files
print("📂 Loading files...")

# 1. Load session list (from NITRC search)
df_sessions = pd.read_csv('tousifo_11_14_2025_8_40_42.csv')

# 2. Load CDR scores (clinical data)
df_cdr = pd.read_csv('OASIS3_UDSb4_cdr.csv')

print(f"✅ Session list loaded: {len(df_sessions)} rows")
print(f"✅ CDR data loaded: {len(df_cdr)} rows")

# Show first few rows of each
print("\n" + "="*70)
print("📋 Session List Preview (first 5 rows):")
print("="*70)
print(df_sessions[['Label', 'Subject', 'M/F', 'Age', 'Scans']].head())

print("\n" + "="*70)
print("📋 CDR Data Preview (first 5 rows):")
print("="*70)
print(df_cdr[['OASISID', 'MMSE', 'CDRSUM', 'CDRTOT', 'dx1']].head())


📂 Loading files...
✅ Session list loaded: 1882 rows
✅ CDR data loaded: 8626 rows

📋 Session List Preview (first 5 rows):
               Label   Subject M/F  Age  \
0  OAS30001_MR_d0129  OAS30001   F   65   
1  OAS30001_MR_d0757  OAS30001   F   67   
2  OAS30001_MR_d2430  OAS30001   F   71   
3  OAS30001_MR_d3132  OAS30001   F   73   
4  OAS30002_MR_d0653  OAS30002   M   68   

                                               Scans  
0                            T1w(2), T2w(2), bold(3)  
1  T1w(2), T2star(1), T2w(2), bold(2), dwi(1), mi...  
2  FLAIR(1), T1w(1), T2star(1), T2w(1), angio(1),...  
3  T1w(1), T2star(1), T2w(2), asl(2), bold(2), dw...  
4                            T1w(2), T2w(2), bold(2)  

📋 CDR Data Preview (first 5 rows):
    OASISID  MMSE  CDRSUM  CDRTOT                 dx1
0  OAS30001  28.0     0.0     0.0  Cognitively normal
1  OAS30001  28.0     0.0     0.0  Cognitively normal
2  OAS30001  30.0     0.0     0.0  Cognitively normal
3  OAS30001  30.0     0.0     0.0  Cog

In [2]:
# Extract Subject ID from CDR data (remove _UDSb4 suffix)
df_cdr['Subject'] = df_cdr['OASISID'].str.split('_').str[0]

# For each subject, get their most recent (latest) diagnosis
df_cdr_latest = df_cdr.sort_values('days_to_visit').groupby('Subject').tail(1)

print(f"📊 Latest CDR data per subject: {len(df_cdr_latest)} subjects")

# Merge CDR scores with session list
df_merged = df_sessions.merge(
    df_cdr_latest[['Subject', 'MMSE', 'CDRSUM', 'CDRTOT', 'dx1']],
    on='Subject',
    how='left'
)

print(f"\n✅ Merged {len(df_merged)} sessions with CDR data")

# Check for missing CDR data
missing_cdr = df_merged['CDRTOT'].isna().sum()
print(f"⚠️  Sessions without CDR data: {missing_cdr}")

# Show diagnosis distribution
print("\n" + "="*70)
print("📈 Diagnosis Distribution:")
print("="*70)
print(df_merged['dx1'].value_counts())

print("\n" + "="*70)
print("📈 CDR Score Distribution:")
print("="*70)
print(df_merged['CDRTOT'].value_counts().sort_index())


📊 Latest CDR data per subject: 1378 subjects

✅ Merged 1882 sessions with CDR data
⚠️  Sessions without CDR data: 1

📈 Diagnosis Distribution:
dx1
Cognitively normal                       1235
AD Dementia                               398
uncertain dementia                        102
DLBD, primary                              21
AD dem w/oth (list B) contribut            12
uncertain, possible NON AD dem             10
DAT                                         9
AD dem Language dysf with                   9
Dementia/PD, primary                        8
AD dem w/CVD not contrib                    8
No dementia                                 8
AD dem w/PDI after AD dem not contrib       7
AD dem Language dysf after                  7
Vascular Demt, primary                      7
Frontotemporal demt. prim                   6
AD dem Language dysf prior                  5
AD dem w/CVD contribut                      3
0.5 in memory only                          3
AD dem w/depresss, not co

In [3]:
# Create binary label: CN (CDR=0) vs AD (CDR >= 0.5)
df_merged['label'] = df_merged['CDRTOT'].apply(
    lambda x: 'CN' if x == 0.0 else ('AD' if x >= 0.5 else 'unknown')
)

# Remove rows with missing CDR or unknown label
df_clean = df_merged[df_merged['label'].isin(['CN', 'AD'])].copy()

print(f"✅ Clean data: {len(df_clean)} sessions")
print(f"   CN: {(df_clean['label'] == 'CN').sum()}")
print(f"   AD: {(df_clean['label'] == 'AD').sum()}")

# Filter to sessions with T1w scans only
df_clean['has_t1w'] = df_clean['Scans'].str.contains('T1w', na=False)
df_with_t1w = df_clean[df_clean['has_t1w'] == True]

print(f"\n✅ Sessions with T1w: {len(df_with_t1w)}")

# Age filter (65-90 years for AD relevance)
df_age_filtered = df_with_t1w[(df_with_t1w['Age'] >= 65) & (df_with_t1w['Age'] <= 90)]

print(f"✅ After age filter (65-90): {len(df_age_filtered)}")
print(f"   CN: {(df_age_filtered['label'] == 'CN').sum()}")
print(f"   AD: {(df_age_filtered['label'] == 'AD').sum()}")

# Limit to 2 visits per subject (to match OASIS-2 structure)
df_sorted = df_age_filtered.sort_values(['Subject', 'Label'])
df_max2 = df_sorted.groupby('Subject').head(2)

print(f"\n✅ After limiting to 2 visits/subject: {len(df_max2)}")
print(f"   Unique subjects: {df_max2['Subject'].nunique()}")
print(f"   CN: {(df_max2['label'] == 'CN').sum()}")
print(f"   AD: {(df_max2['label'] == 'AD').sum()}")


✅ Clean data: 1881 sessions
   CN: 1243
   AD: 638

✅ Sessions with T1w: 1877
✅ After age filter (65-90): 1428
   CN: 843
   AD: 585

✅ After limiting to 2 visits/subject: 1206
   Unique subjects: 852
   CN: 692
   AD: 514


In [4]:
# Set random seed for reproducibility
np.random.seed(42)

# Get unique subjects for each class
cn_subjects = df_max2[df_max2['label'] == 'CN']['Subject'].unique()
ad_subjects = df_max2[df_max2['label'] == 'AD']['Subject'].unique()

print(f"Available subjects:")
print(f"  CN: {len(cn_subjects)}")
print(f"  AD: {len(ad_subjects)}")

# Sample 150 subjects from each group
cn_selected = np.random.choice(cn_subjects, size=150, replace=False)
ad_selected = np.random.choice(ad_subjects, size=150, replace=False)

print(f"\n✅ Selected subjects:")
print(f"  CN: 150")
print(f"  AD: 150")
print(f"  Total: 300 subjects")

# Get all sessions for selected subjects (max 2 visits each)
selected_subjects = list(cn_selected) + list(ad_selected)
df_final = df_max2[df_max2['Subject'].isin(selected_subjects)]

print(f"\n✅ Final dataset:")
print(f"  Total sessions: {len(df_final)}")
print(f"  CN sessions: {(df_final['label'] == 'CN').sum()}")
print(f"  AD sessions: {(df_final['label'] == 'AD').sum()}")
print(f"  Average visits per subject: {len(df_final) / 300:.2f}")

# Gender balance check
print(f"\n⚖️  Gender distribution:")
print(df_final['M/F'].value_counts())

# Age statistics
print(f"\n📊 Age statistics:")
print(f"  Mean age: {df_final['Age'].mean():.1f} years")
print(f"  Age range: {df_final['Age'].min():.0f} - {df_final['Age'].max():.0f}")

print(f"\n💾 Estimated download size: ~{len(df_final) * 0.04:.1f} GB (T1w only)")


Available subjects:
  CN: 464
  AD: 388

✅ Selected subjects:
  CN: 150
  AD: 150
  Total: 300 subjects

✅ Final dataset:
  Total sessions: 423
  CN sessions: 225
  AD sessions: 198
  Average visits per subject: 1.41

⚖️  Gender distribution:
M/F
F    225
M    198
Name: count, dtype: int64

📊 Age statistics:
  Mean age: 74.1 years
  Age range: 65 - 90

💾 Estimated download size: ~16.9 GB (T1w only)


In [7]:
from google.colab import files
import os

print("📦 Creating download files...")

# Re-create the files (in case variables were lost)
# Save full dataset
df_final.to_csv('oasis3_filtered_300subjects.csv', index=False)

# Save session IDs
session_ids = df_final['Label'].tolist()
with open('oasis3_download_list.txt', 'w') as f:
    for sid in session_ids:
        f.write(sid + '\n')

# Save summary
summary = {
    'total_subjects': 300,
    'total_sessions': int(len(df_final)),
    'cn_sessions': int((df_final['label'] == 'CN').sum()),
    'ad_sessions': int((df_final['label'] == 'AD').sum()),
    'mean_age': float(df_final['Age'].mean()),
    'estimated_size_gb': float(len(df_final) * 0.04)
}
import json
with open('oasis3_dataset_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

# Verify files exist
print("\n✅ Files created:")
for fname in ['oasis3_download_list.txt', 'oasis3_filtered_300subjects.csv', 'oasis3_dataset_summary.json']:
    if os.path.exists(fname):
        size = os.path.getsize(fname)
        print(f"   {fname:40s} ({size:,} bytes)")

# Download all 3 files
print("\n📥 Downloading files to your computer...")
print("(3 download dialogs will open)")
print()

files.download('oasis3_download_list.txt')
files.download('oasis3_filtered_300subjects.csv')
files.download('oasis3_dataset_summary.json')

print("\n✅ All files downloaded successfully!")


📦 Creating download files...

✅ Files created:
   oasis3_download_list.txt                 (7,614 bytes)
   oasis3_filtered_300subjects.csv          (63,571 bytes)
   oasis3_dataset_summary.json              (159 bytes)

📥 Downloading files to your computer...
(3 download dialogs will open)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ All files downloaded successfully!
